# 03_compute_bandSNR

In [1]:
import numpy as np
from os.path import join as pjoin
from os.path import isdir
import os
import matplotlib.pyplot as plt
from matplotlib import cm, colors
import mne_bids
import mne
from mne_bids import write_raw_bids, BIDSPath
from scipy import stats
import re
from scipy import signal
import pandas as pd
from scipy import signal, fftpack

In [2]:
def load_sub_raw_data(bids_root, subject_idx, run_idx):
    """
    load raw meg data. 
    """
    
    if not isinstance(subject_idx, str):
        raise ValueError('subject_dix must be str')
        
    if not isinstance(run_idx, str):
        raise ValueError('run_idx must be str')
    
    subject_data_folder = pjoin(bids_root, 'sub-{}'.format(subject_idx), 'ses-movie', 'meg')
    fname = 'sub-' + subject_idx + '_ses-movie_task-movie_run-' + run_idx + '_meg.ds'
    raw_data_path = pjoin(subject_data_folder, fname)
    raw_data = mne.io.read_raw_ctf(raw_data_path, preload='True')
    
    print('total channels number is {}'.format(len(raw_data.info['chs'])))
    print('sample frequency is {} Hz'.format(raw_data.info['sfreq']))

    return raw_data


In [3]:
def load_pre_megdata(bids_root, sub, run):
    """
    load raw meg data
    """
    
    sub_path = BIDSPath(subject=sub, run=int(run), task='movie', session='movie', root=bids_root)
    raw_sub_ev = mne_bids.read_raw_bids(sub_path)
    raw_sub = load_sub_raw_data(bids_root, subject_idx=sub, run_idx=run)
    ch_name_picks = mne.pick_channels_regexp(raw_sub.ch_names, regexp='M[LRZ]...-4503')
    type_picks = mne.pick_types(raw_sub.info, meg=True)
    sub_picks= np.intersect1d(ch_name_picks, type_picks)
    sub_raw_data = raw_sub.get_data(picks=sub_picks)
    events_sub = mne.events_from_annotations(raw_sub_ev)
    sample_sub = events_sub[0][:,0]
    
    sub_data = sub_raw_data.take(sample_sub, axis=1)
    return sub_data

def load_post_megdata(bids_root, sub, run):
    """
    load preprocessed meg data
    """
    
    sub_path = BIDSPath(subject=sub, run=int(run), task='movie', session='movie', processing='preproc', root=bids_root)
    raw_sub = mne_bids.read_raw_bids(sub_path)

    ch_name_picks = mne.pick_channels_regexp(raw_sub.ch_names, regexp='M[LRZ]...-4503')
    type_picks = mne.pick_types(raw_sub.info, meg=True)
    sub_picks= np.intersect1d(ch_name_picks, type_picks)
    sub_raw_data = raw_sub.get_data(picks=sub_picks)
    events_sub = mne.events_from_annotations(raw_sub)
    sample_sub = events_sub[0][:,0]
    
    sub_data = sub_raw_data.take(sample_sub, axis=1)
    return sub_data

In [4]:
def bandSNR(data, fs):
    """
    Calculate bandSNR for 
    Parameters
    ----------
    data: array
        [n_channels, n_samples]
        
    Returns
    -------
    bandSNR : array
        [n_channels, n_bands]
    """
    
    # remove linear trend
    data_detrend = signal.detrend(data, axis=-1)

    # convert to frequency domain        
    freqs, psd = signal.welch(data_detrend, fs=fs)

    band = [[1, 4], [4, 8], [8, 13], [13, 30], [30, 100]]
    # "delta": 1-4Hz
    # "theta": 4-8Hz
    # "alpha": 8-13Hz
    # "beta": 13-30Hz
    # "gamma": 30-100Hz

    bandSNR = [np.sum(psd[:, (freqs>i[0]) * (freqs<i[1])], axis=-1) / np.sum(psd[:, freqs<0.5*fs], axis=-1) for i in band]
    bandSNR = np.asarray(bandSNR).T
    
    return bandSNR

In [5]:
# define variables
sub_list = ['{0:0>2d}'.format(sub) for sub in np.arange(1,12)]
run_list = ['{0:0>2d}'.format(run) for run in np.arange(1,9)]
band = ['delta', 'theta', 'alpha', 'beta', 'gamma']

# set path
pre_root = '/nfs/e5/studyforrest/forrest_movie_meg'
post_root = '/nfs/e5/studyforrest/forrest_movie_meg/meg_preproc_data_bak/'

results_pth = pjoin(os.getcwd(),'tech_val_results')
if os.path.exists(results_pth) is False:
    os.mkdir(results_pth)

In [6]:
# compute bandSNR
pre_bandSNR_data = {}
post_bandSNR_data = {}

for sub in sub_list:
    pre_bandSNR_data[sub] = []
    post_bandSNR_data[sub] = []

    # get runlist
    if sub == '01':
        run_ls = run_list + ['09']
    else:
        run_ls = run_list
        
    for run in run_ls:       
        # compute bandSNR for raw meg data
        pre_data = load_pre_megdata(pre_root, sub=sub, run=run)
        pre_bandSNR = bandSNR(pre_data, fs=600)
        pre_bandSNR_data[sub].append(pre_bandSNR)
        
        # compute bandSNR for preprocessed meg data
        post_data = load_post_megdata(post_root, sub=sub, run=run)
        post_bandSNR = bandSNR(post_data, fs=600)
        post_bandSNR_data[sub].append(post_bandSNR)

        print('sub-{0} run-{1} done'.format(sub, run))

ds directory : /nfs/e5/studyforrest/forrest_movie_meg/sub-01/ses-movie/meg/sub-01_ses-movie_task-movie_run-01_meg.ds
    res4 data read.
    hc data read.
    Separate EEG position data file read.
    Quaternion matching (desired vs. transformed):
       4.65   74.88    0.00 mm <->    4.65   74.88    0.00 mm (orig :  -65.68   46.24 -249.17 mm) diff =    0.000 mm
      -4.65  -74.88    0.00 mm <->   -4.65  -74.88   -0.00 mm (orig :   42.00  -58.24 -250.44 mm) diff =    0.000 mm
      92.94    0.00    0.00 mm <->   92.94   -0.00    0.00 mm (orig :   46.46   62.07 -225.18 mm) diff =    0.000 mm
    Coordinate transformations established.
    Polhemus data for 3 HPI coils added
    Device coordinate locations for 3 HPI coils added
    Measurement info composed.
Finding samples for /nfs/e5/studyforrest/forrest_movie_meg/sub-01/ses-movie/meg/sub-01_ses-movie_task-movie_run-01_meg.ds/sub-01_ses-movie_task-movie_run-01_meg.meg4: 
    System clock channel is available, checking which samples ar

      -4.19  -74.86    0.00 mm <->   -4.19  -74.86    0.00 mm (orig :   42.69  -58.14 -249.36 mm) diff =    0.000 mm
      92.78    0.00    0.00 mm <->   92.78   -0.00    0.00 mm (orig :   44.87   62.49 -228.12 mm) diff =    0.000 mm
    Coordinate transformations established.
    Polhemus data for 3 HPI coils added
    Device coordinate locations for 3 HPI coils added
    Measurement info composed.
Finding samples for /nfs/e5/studyforrest/forrest_movie_meg/sub-01/ses-movie/meg/sub-01_ses-movie_task-movie_run-03_meg.ds/sub-01_ses-movie_task-movie_run-03_meg.meg4: 
    System clock channel is available, checking which samples are valid.
    3 x 186000 = 558000 samples from 409 chs
Current compensation grade : 3
Reading 0 ... 557999  =      0.000 ...   929.998 secs...
total channels number is 409
sample frequency is 600.0 Hz
Used Annotations descriptions: ['1', '2', '3']
Opening raw data file /nfs/e5/studyforrest/forrest_movie_meg/meg_preproc_data_bak/sub-01/ses-movie/meg/sub-01_ses-movi

    res4 data read.
    hc data read.
    Separate EEG position data file read.
    Quaternion matching (desired vs. transformed):
       4.42   75.26    0.00 mm <->    4.42   75.26    0.00 mm (orig :  -63.71   50.33 -254.15 mm) diff =    0.000 mm
      -4.42  -75.26    0.00 mm <->   -4.42  -75.26    0.00 mm (orig :   40.61  -58.50 -251.43 mm) diff =    0.000 mm
      92.82    0.00    0.00 mm <->   92.82   -0.00   -0.00 mm (orig :   47.54   60.85 -222.66 mm) diff =    0.000 mm
    Coordinate transformations established.
    Polhemus data for 3 HPI coils added
    Device coordinate locations for 3 HPI coils added
    Measurement info composed.
Finding samples for /nfs/e5/studyforrest/forrest_movie_meg/sub-01/ses-movie/meg/sub-01_ses-movie_task-movie_run-06_meg.ds/sub-01_ses-movie_task-movie_run-06_meg.meg4: 
    System clock channel is available, checking which samples are valid.
    3 x 184200 = 552600 samples from 409 chs
Current compensation grade : 3
Reading events from /nfs/e5/stud

/nfs/s2/userhome/liuxingyu/.local/lib/python3.8/site-packages/mne_bids/path.py:1065: RuntimeWarning: Did not find any events.tsv associated with sub-01_ses-movie_task-movie_run-08.

The search_str was "/nfs/e5/studyforrest/forrest_movie_meg/sub-01/**/sub-01_ses-movie*events.tsv"
  warn(msg)


Reading channel info from /nfs/e5/studyforrest/forrest_movie_meg/sub-01/ses-movie/meg/sub-01_ses-movie_task-movie_run-08_channels.tsv.
ds directory : /nfs/e5/studyforrest/forrest_movie_meg/sub-01/ses-movie/meg/sub-01_ses-movie_task-movie_run-08_meg.ds
    res4 data read.
    hc data read.
    Separate EEG position data file read.
    Quaternion matching (desired vs. transformed):
       4.06   76.10    0.00 mm <->    4.06   76.10    0.00 mm (orig :  -63.46   51.82 -257.20 mm) diff =    0.000 mm
      -4.06  -76.10    0.00 mm <->   -4.06  -76.10   -0.00 mm (orig :   41.34  -58.87 -257.04 mm) diff =    0.000 mm
      92.82    0.00    0.00 mm <->   92.82    0.00    0.00 mm (orig :   48.65   59.86 -225.01 mm) diff =    0.000 mm
    Coordinate transformations established.
    Polhemus data for 3 HPI coils added
    Device coordinate locations for 3 HPI coils added
    Measurement info composed.
Finding samples for /nfs/e5/studyforrest/forrest_movie_meg/sub-01/ses-movie/meg/sub-01_ses-movie_

ZeroDivisionError: integer division or modulo by zero

In [14]:
sub = '01'
run = '08'
bids_root = '/nfs/e5/studyforrest/forrest_movie_meg'
sub_path = BIDSPath(subject=sub, run=int(run), task='movie', session='movie', root=bids_root)
raw_sub_ev = mne_bids.read_raw_bids(sub_path)

ds directory : /nfs/e5/studyforrest/forrest_movie_meg/sub-01/ses-movie/meg/sub-01_ses-movie_task-movie_run-08_meg.ds
    res4 data read.
    hc data read.
    Separate EEG position data file read.
    Quaternion matching (desired vs. transformed):
       4.06   76.10    0.00 mm <->    4.06   76.10    0.00 mm (orig :  -63.46   51.82 -257.20 mm) diff =    0.000 mm
      -4.06  -76.10    0.00 mm <->   -4.06  -76.10   -0.00 mm (orig :   41.34  -58.87 -257.04 mm) diff =    0.000 mm
      92.82    0.00    0.00 mm <->   92.82    0.00    0.00 mm (orig :   48.65   59.86 -225.01 mm) diff =    0.000 mm
    Coordinate transformations established.
    Polhemus data for 3 HPI coils added
    Device coordinate locations for 3 HPI coils added
    Measurement info composed.
Finding samples for /nfs/e5/studyforrest/forrest_movie_meg/sub-01/ses-movie/meg/sub-01_ses-movie_task-movie_run-08_meg.ds/sub-01_ses-movie_task-movie_run-08_meg.meg4: 
    System clock channel is available, checking which samples ar

/nfs/s2/userhome/liuxingyu/.local/lib/python3.8/site-packages/mne_bids/path.py:1065: RuntimeWarning: Did not find any events.tsv associated with sub-01_ses-movie_task-movie_run-08.

The search_str was "/nfs/e5/studyforrest/forrest_movie_meg/sub-01/**/sub-01_ses-movie*events.tsv"
  warn(msg)


In [ ]:

raw_sub_ev = mne_bids.read_raw_bids(sub_path)
raw_sub = load_sub_raw_data(bids_root, subject_idx=sub, run_idx=run)
ch_name_picks = mne.pick_channels_regexp(raw_sub.ch_names, regexp='M[LRZ]...-4503')
type_picks = mne.pick_types(raw_sub.info, meg=True)
sub_picks= np.intersect1d(ch_name_picks, type_picks)
sub_raw_data = raw_sub.get_data(picks=sub_picks)
events_sub = mne.events_from_annotations(raw_sub_ev)
sample_sub = events_sub[0][:,0]

sub_data = sub_raw_data.take(sample_sub, axis=1)

In [ ]:
# save bandSNR
for sub in sub_list[1:]:
    pre_bandSNR_data[sub].append(np.nan)
    post_bandSNR_data[sub].append(np.nan)
pre_df = pd.DataFrame(pre_bandSNR_data, columns=sub_list, index=run_list+['09'])
post_df = pd.DataFrame(post_bandSNR_data, columns=sub_list, index=run_list+['09'])
pre_df.to_pickle(pjoin(results_pth, 'pre_bandSNR_data.pickle'))
post_df.to_pickle(pjoin(results_pth, 'post_bandSNR_data.pickle'))